In [1]:
import math
import os
import subprocess

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

#
from utilist_feature_engineering import *

# some settings for displaying Pandas results
pd.set_option('display.width', 2000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.precision', 4)
pd.set_option('display.max_colwidth', -1)

ModuleNotFoundError: No module named 'utilist_feature_engineering'

# Load Data

In [ ]:
# load train/test data
path = r'D:\NEU\Năm 3\DATA PREP\PROJECT_DATAPREP\GITHUB REPO\dataset\dseb63_final_project_DP_dataset\\'

# train
application_train = pd.read_csv(path + 'dseb63_' + 'application_train.csv')
application_train.drop(columns='Unnamed: 0', inplace=True)
# test
application_test = pd.read_csv(path + 'dseb63_' + 'application_test.csv')
application_test.drop(columns='Unnamed: 0', inplace=True)

# filter by tvt code
application_tvt_extend = pd.read_pickle(
    "application_tvt_extend.pkl", compression="bz2")
application_train_filtered = (application_tvt_extend.query("tvt_code == 'train'")
                              .merge(application_train[["SK_ID_CURR"]], on="SK_ID_CURR")
                              .drop(columns=["tvt_code"]))
application_train_filtered.head()

In [ ]:
# load previous application
data_path = "home-credit-default-risk/POS_CASH_balance.csv"
pos_cash = pd.read_csv(path + 'dseb63_' + 'POS_CASH_balance.csv')
print(pos_cash.shape)
pos_cash.head()

(8543375, 8)


,SK_ID_PREV,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,SK_ID_CURR
0,1803195,-31,48.0,45.0,Active,0,0,185279
1,1803195,-17,48.0,31.0,Active,0,0,185279
2,1803195,-21,48.0,35.0,Active,0,0,185279
3,1803195,-8,48.0,21.0,Active,0,0,185279
4,1803195,-4,48.0,17.0,Active,0,0,185279


In [ ]:
# filter here for look up application with years
pos_cash["MONTHS_BALANCE"] = pos_cash["MONTHS_BALANCE"] * -1
print(pos_cash.shape)

(8543375, 8)


## DPD handling

Day Past Due : Số ngày quá hạn


In [ ]:
# tạo feature đánh dấu client đã trễ hạn
pos_cash["is_DPD"] = (pos_cash["SK_DPD"] > 0).astype('int64')
pos_cash["is_DPD_DEF"] = (pos_cash["SK_DPD_DEF"] > 0).astype('int64')

# drop columns cũ
pos_cash.drop(columns=["SK_DPD", "SK_DPD_DEF"], inplace=True)
pos_cash

,SK_ID_PREV,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_ID_CURR,is_DPD,is_DPD_DEF
0,1803195,31,48.0,45.0,Active,185279,0,0
1,1803195,17,48.0,31.0,Active,185279,0,0
2,1803195,21,48.0,35.0,Active,185279,0,0
3,1803195,8,48.0,21.0,Active,185279,0,0
4,1803195,4,48.0,17.0,Active,185279,0,0
...,...,...,...,...,...,...,...,...
8543370,2340627,2,NaN,NaN,Signed,284649,0,0
8543371,1011796,2,6.0,6.0,Active,69172,0,0
8543372,1298851,1,12.0,12.0,Active,151122,0,0
8543373,1550592,1,1.0,0.0,Completed,56549,0,0


## Categorical features

In [ ]:
# get list categorical attributes
list_cate = pos_cash.dtypes[pos_cash.dtypes == 'object'].index.tolist()
list_cate

['NAME_CONTRACT_STATUS']

In [ ]:
# construct categorical mapping
dict_onehot = {}
for cate in list_cate:
    list_val = pos_cash[cate].value_counts().index.tolist()
    dict_onehot[cate] = list_val

In [ ]:
dict_onehot

{'NAME_CONTRACT_STATUS': ['Active',
  'Completed',
  'Signed',
  'Demand',
  'Returned to the store',
  'Approved',
  'Amortized debt',
  'Canceled',
  'XNA']}

## One hot

In [ ]:
%%time
df_onehot = gen_one_hot_feat(pos_cash, dict_onehot, main_key="SK_ID_CURR")
print(df_onehot.shape)

(8543375, 10)
Wall time: 20.9 s


In [ ]:
%%time
application_train_filtered = application_train[['SK_ID_CURR','TARGET']]

df_agg01 = agg_common_data(df_onehot, ["max", "sum", "mean"], main_key="SK_ID_CURR")
eval_agg01 = feature_evaluate(application_train_filtered, df_agg01)
display(eval_agg01)

{'NAME_CONTRACT_STATUS_Active': ['max', 'sum', 'mean'],
 'NAME_CONTRACT_STATUS_Completed': ['max', 'sum', 'mean'],
 'NAME_CONTRACT_STATUS_Signed': ['max', 'sum', 'mean'],
 'NAME_CONTRACT_STATUS_Demand': ['max', 'sum', 'mean'],
 'NAME_CONTRACT_STATUS_Returned_to_the_store': ['max', 'sum', 'mean'],
 'NAME_CONTRACT_STATUS_Approved': ['max', 'sum', 'mean'],
 'NAME_CONTRACT_STATUS_Amortized_debt': ['max', 'sum', 'mean'],
 'NAME_CONTRACT_STATUS_Canceled': ['max', 'sum', 'mean'],
 'NAME_CONTRACT_STATUS_XNA': ['max', 'sum', 'mean']}

After agg: (289444, 27)


,name,auc,corr,coverage
1,NAME_CONTRACT_STATUS_Active_sum,0.5460,-3.6724e-02,1.0
4,NAME_CONTRACT_STATUS_Completed_sum,0.5406,-2.0064e-02,1.0
3,NAME_CONTRACT_STATUS_Completed_max,0.5116,-2.0230e-02,1.0
2,NAME_CONTRACT_STATUS_Active_mean,0.5082,-8.5425e-03,1.0
7,NAME_CONTRACT_STATUS_Signed_sum,0.5036,-4.7866e-03,1.0
6,NAME_CONTRACT_STATUS_Signed_max,0.5033,-4.5674e-03,1.0
5,NAME_CONTRACT_STATUS_Completed_mean,0.5023,7.0014e-04,1.0
14,NAME_CONTRACT_STATUS_Returned_to_the_store_mean,0.5015,1.1307e-02,1.0
13,NAME_CONTRACT_STATUS_Returned_to_the_store_sum,0.5015,8.0469e-03,1.0
12,NAME_CONTRACT_STATUS_Returned_to_the_store_max,0.5015,6.9723e-03,1.0


Wall time: 4.52 s


In [ ]:
eval_agg01[eval_agg01['auc'] <= 0.501].shape

(16, 4)

In [ ]:
sel_feat = eval_agg01[eval_agg01['auc'] > 0.501]["name"].tolist()
df_agg01 = df_agg01[sel_feat]
print(df_agg01.shape)

(289444, 11)


## Numerical Features

In [ ]:
series_type = pos_cash.dtypes
list_num = series_type[series_type == "int64"]
list_num = [cname for cname in list_num.index if cname not in ["SK_ID_PREV", "SK_ID_CURR"]]
list_num

['MONTHS_BALANCE', 'is_DPD', 'is_DPD_DEF']

In [ ]:
%%time
df_agg02 = agg_common_data(pos_cash[["SK_ID_CURR"] + list_num], ["max", "min", "sum", "mean", "std"], main_key="SK_ID_CURR")
eval_agg02 = feature_evaluate(application_train_filtered, df_agg02)
display(eval_agg02)

{'MONTHS_BALANCE': ['max', 'min', 'sum', 'mean', 'std'],
 'is_DPD': ['max', 'min', 'sum', 'mean', 'std'],
 'is_DPD_DEF': ['max', 'min', 'sum', 'mean', 'std']}

After agg: (289444, 15)


,name,auc,corr,coverage
4,MONTHS_BALANCE_std,0.5571,-0.0539,0.9988
0,MONTHS_BALANCE_max,0.5561,-0.0532,1.0000
2,MONTHS_BALANCE_sum,0.5535,-0.0408,1.0000
3,MONTHS_BALANCE_mean,0.5411,-0.0323,1.0000
14,is_DPD_DEF_std,0.5274,0.0509,0.9988
13,is_DPD_DEF_mean,0.5273,0.0480,1.0000
9,is_DPD_std,0.5265,0.0410,0.9988
8,is_DPD_mean,0.5264,0.0296,1.0000
12,is_DPD_DEF_sum,0.5258,0.0263,1.0000
10,is_DPD_DEF_max,0.5255,0.0406,1.0000


Wall time: 2.87 s


## Continuous features

In [ ]:
# get list continuous attributes
list_con = pos_cash.dtypes[pos_cash.dtypes == 'float64'].index.tolist()
list_con

['CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE']

In [ ]:
df_con = pos_cash[["SK_ID_PREV", "SK_ID_CURR"] + list_con].copy()
df_con.head()

,SK_ID_PREV,SK_ID_CURR,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE
0,1803195,185279,48.0,45.0
1,1803195,185279,48.0,31.0
2,1803195,185279,48.0,35.0
3,1803195,185279,48.0,21.0
4,1803195,185279,48.0,17.0


In [ ]:
%%time
df_agg03 = agg_common_data(df_con[["SK_ID_CURR"] + list_con], ["max", "min", "sum", "mean", "std"], main_key="SK_ID_CURR")
eval_agg03 = feature_evaluate(application_train_filtered, df_agg03)
display(eval_agg03)

{'CNT_INSTALMENT': ['max', 'min', 'sum', 'mean', 'std'],
 'CNT_INSTALMENT_FUTURE': ['max', 'min', 'sum', 'mean', 'std']}

After agg: (289444, 10)


,name,auc,corr,coverage
2,CNT_INSTALMENT_sum,0.5261,-0.0156,1.0000
1,CNT_INSTALMENT_min,0.5218,0.0197,0.9999
7,CNT_INSTALMENT_FUTURE_sum,0.5204,-0.0069,1.0000
8,CNT_INSTALMENT_FUTURE_mean,0.5195,0.0270,0.9999
3,CNT_INSTALMENT_mean,0.5152,0.0174,0.9999
4,CNT_INSTALMENT_std,0.5142,0.0036,0.9987
6,CNT_INSTALMENT_FUTURE_min,0.5100,0.0198,0.9999
9,CNT_INSTALMENT_FUTURE_std,0.5065,0.0153,0.9987
5,CNT_INSTALMENT_FUTURE_max,0.5055,0.0127,0.9999
0,CNT_INSTALMENT_max,0.5052,0.0126,0.9999


Wall time: 1.9 s


# Save Features

In [ ]:
df_feat = df_agg01.join(df_agg02).join(df_agg03)
print(df_feat.shape)

(289444, 36)


In [ ]:
%%time
fname = "pos_cash"


fname = os.path.join("features", "{}.pkl.bz2".format(fname))
df_feat.to_pickle(fname, compression="bz2")
print("Store features completed!")

Store features completed!
Wall time: 3.05 s


In [ ]:
import pickle
import bz2

filename = "features/pos_cash.pkl.bz2"
with bz2.open(filename, 'rb') as f:
    df_out = pickle.load(f)

print(df_out.shape)
df_out

(289444, 36)


,NAME_CONTRACT_STATUS_Active_sum,NAME_CONTRACT_STATUS_Completed_sum,NAME_CONTRACT_STATUS_Completed_max,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Signed_sum,NAME_CONTRACT_STATUS_Signed_max,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Returned_to_the_store_mean,NAME_CONTRACT_STATUS_Returned_to_the_store_sum,NAME_CONTRACT_STATUS_Returned_to_the_store_max,NAME_CONTRACT_STATUS_Signed_mean,MONTHS_BALANCE_max,MONTHS_BALANCE_min,MONTHS_BALANCE_sum,MONTHS_BALANCE_mean,MONTHS_BALANCE_std,is_DPD_max,is_DPD_min,is_DPD_sum,is_DPD_mean,is_DPD_std,is_DPD_DEF_max,is_DPD_DEF_min,is_DPD_DEF_sum,is_DPD_DEF_mean,is_DPD_DEF_std,CNT_INSTALMENT_max,CNT_INSTALMENT_min,CNT_INSTALMENT_sum,CNT_INSTALMENT_mean,CNT_INSTALMENT_std,CNT_INSTALMENT_FUTURE_max,CNT_INSTALMENT_FUTURE_min,CNT_INSTALMENT_FUTURE_sum,CNT_INSTALMENT_FUTURE_mean,CNT_INSTALMENT_FUTURE_std
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,12,3,1,0.8000,0,0,0.2000,0.0,0,0,0.0000,15,1,112,7.4667,4.8087,0,0,0,0.0000,0.0000,0,0,0,0.0000,0.0000,12.0,2.0,122.0,8.1333,3.7582,12.0,0.0,96.0,6.4000,4.5166
1,21,2,1,0.9130,0,0,0.0870,0.0,0,0,0.0000,53,12,738,32.0870,15.9428,1,0,2,0.0870,0.2881,0,0,0,0.0000,0.0000,36.0,10.0,517.0,22.4783,13.2386,36.0,0.0,396.0,17.2174,13.8956
2,31,3,1,0.9118,0,0,0.0882,0.0,0,0,0.0000,82,23,1875,55.1471,21.8176,0,0,0,0.0000,0.0000,0,0,0,0.0000,0.0000,12.0,4.0,292.0,8.5882,3.0163,12.0,0.0,145.0,4.2647,3.2873
3,11,3,1,0.7857,0,0,0.2143,0.0,0,0,0.0000,24,5,174,12.4286,6.2722,0,0,0,0.0000,0.0000,0,0,0,0.0000,0.0000,12.0,2.0,139.0,9.9286,3.5184,12.0,0.0,111.0,7.9286,4.4456
4,7,0,0,1.0000,0,0,0.0000,0.0,0,0,0.0000,79,73,532,76.0000,2.1602,0,0,0,0.0000,0.0000,0,0,0,0.0000,0.0000,6.0,6.0,42.0,6.0000,0.0000,6.0,0.0,21.0,3.0000,2.1602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307505,18,2,1,0.9000,0,0,0.1000,0.0,0,0,0.0000,68,40,1089,54.4500,10.1488,0,0,0,0.0000,0.0000,0,0,0,0.0000,0.0000,10.0,8.0,182.0,9.1000,1.0208,10.0,0.0,91.0,4.5500,3.0345
307506,43,4,1,0.9149,0,0,0.0851,0.0,0,0,0.0000,82,2,1304,27.7447,24.3288,1,0,4,0.0851,0.2821,1,0,4,0.0851,0.2821,12.0,6.0,516.0,10.9787,2.2793,12.0,0.0,255.0,5.4255,3.7806
307508,43,2,1,0.9348,1,1,0.0435,0.0,0,0,0.0217,75,1,1337,29.0652,25.7072,0,0,0,0.0000,0.0000,0,0,0,0.0000,0.0000,12.0,10.0,500.0,11.1111,1.0050,12.0,0.0,265.0,5.8889,3.3993
